In [1]:
import json
import numpy as np
import pandas as pd

In [4]:
names = dict()
with open('../data/heroes.json') as f:
    data = json.load(f)
    for i in data:
        names[i["id"]] = i['localized_name']

In [ ]:
df = pd.DataFrame.from_csv("../data/liquid_picks_bans.csv")
df.head()